## -- Notebook: 01_bronze_layer.ipynb
-- Camada: Bronze
-- Objetivo: Ingerir dados brutos da fonte (trips_raw) para a camada Bronze.

## -- 1. Criar a Tabela Delta na Camada Bronze
-- Usaremos CREATE TABLE AS SELECT (CTAS) para criar a tabela bronze
-- a partir da nossa "fonte" (mc_labs.trips_raw).
-- No mundo real, esta fonte poderia ser um arquivo CSV/JSON em um blob storage,
-- um tópico Kafka, um banco de dados relacional, etc.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mc_labs.trips_bronze
USING DELTA
AS SELECT
tpep_pickup_datetime,
tpep_dropoff_datetime,
trip_distance,
fare_amount,
pickup_zip,
dropoff_zip,
current_timestamp() AS ingestion_timestamp, -- Adiciona um timestamp de ingestão
input_file_name() AS source_file_name       -- Adiciona o nome do arquivo de origem (se aplicável)
FROM mc_labs.trips_raw;

## -- 2. Verificar o Esquema e os Dados da Tabela Bronze

In [0]:
%sql
DESCRIBE DETAIL mc_labs.trips_bronze;


In [0]:
%sql
DESCRIBE HISTORY mc_labs.trips_bronze;


In [0]:
%sql
SELECT * FROM mc_labs.trips_bronze LIMIT 10;

## -- 3. Simular uma Ingestão Incremental (Novos Dados)
-- No cenário real, novos dados chegariam e seriam adicionados à tabela Bronze.
-- Vamos simular isso inserindo alguns registros de exemplo.
-- Para uma ingestão incremental mais robusta (com atualizações), usaríamos MERGE INTO.
-- Para a camada Bronze, muitas vezes é um simples APPEND (INSERT INTO).

In [0]:
%sql
INSERT INTO mc_labs.trips_bronze VALUES
('2023-01-01 10:00:00', '2023-01-01 10:15:00', 2.5, 15.00, 10001, 10002, current_timestamp(), 'simulated_new_data_1.csv'),
('2023-01-01 11:00:00', '2023-01-01 11:30:00', 5.0, 25.50, 10003, 10004, current_timestamp(), 'simulated_new_data_2.csv');

-- Verificar os novos dados e o histórico
SELECT * FROM mc_labs.trips_bronze WHERE pickup_zip IN (10001, 10003);

In [0]:
%sql
DESCRIBE HISTORY mc_labs.trips_bronze; -- Observe uma nova versão no histórico

## -- 4. Exemplo de Time Travel na Camada Bronze
-- Podemos consultar o estado da tabela antes da inserção incremental
-- (Assumindo que a primeira operação de CREATE TABLE AS SELECT foi a versão 0)

In [0]:
%sql
SELECT * FROM mc_labs.trips_bronze VERSION AS OF 0 LIMIT 10;

In [0]:
%sql
SELECT MAX(version) - 1 AS previous_version
FROM (DESCRIBE HISTORY mc_labs.trips_bronze);